In [1]:
import hashlib
import numpy as np
import random
import pandas as pd

# Count min implementation

## Matrix creation
    - Creates the array of counters of width w and depth d ( TODO : Change this values and see which are better )
    - Creates array of hash functions with size d
## Update matrix
    - For the new value it implements the hashes and stores the hashcode%m in the matrix

In [2]:
def matrixCreation(w,d):
    matrix = np.zeros([d, w], dtype=np.int32)
    return matrix

In [3]:
def update(matrix,x):
    for i in range(d):
        value = int(hash_functions[i](x.encode('utf-8')).hexdigest(), 16) % 10**8
        matrix[i][value % w] += 1
        print(value % w)

In [4]:
def queryCountMin(matrix, x):
    minimum =999
    for i in range(d):
        value = int(hash_functions[i](x.encode('utf-8')).hexdigest(), 16) % 10**8
        if matrix[i][value % w] < minimum:
            minimum = matrix[i][value % w]
    return minimum

- Maintain an matrix of (d x w) hash buckets
        - where d is the number of hash functions and and w is the range of hash values. The width w is determined by ε and the depth d is determined by δ Each bucket contains a counter.
- As new item arrives, we apply each hash function and increment the corresponding hash bucket counter. We increment d counters.
- To find the frequency count for an item, we apply the d hash functions and read the counter values from the corresponding buckets.  We end up with d  counter values. We take the minimum of the counter values and return that as the estimated count.

In [5]:
# TODO - these are not hash functions
#len(hash_functions)
def hash_function(d):
    vec = []
    vec.append(hashlib.md5)
    vec.append(hashlib.sha1)
    #for i in range(d):
     #   m = hashlib.md5()
      #  vec.append(m)
    return vec

In [6]:
d=2
hash_functions = hash_function(d)


In [7]:
int(hash_functions[0]('str(192.192)'.encode('utf-8')).hexdigest(),16)%10**8
#h2 = int(hashlib.md5(str(split_packet[3]).encode('utf-8')).hexdigest(),16)%10**8


67498398

In [8]:
CTU_NETFLOW_FILEPATH ="capture20110812.pcap.netflow.labeled"

COLUMNS = ["Date flow start","Durat","Prot","Src IP Addr:Port","Dst IP Addr:Port","Flags","Tos","Packets Bytes","Flows","Label","Labels"]
print(len(COLUMNS))

11


In [24]:
def determineAcc (df):
    trueValues = df.groupby(['Dst IP Addr:Port']).size()
    trueValues = trueValues.reset_index()
    trueValues.columns = ['IP','True Values']
    comparisonDataFrame = trueValues
    comparisonDataFrame['countmin'] = comparisonDataFrame.apply(lambda row: queryCountMin(counterMatrix,str(row["IP"])),axis=1)

    comparisonDataFrame['countminAcc'] =  comparisonDataFrame.apply(lambda row: True if row['True Values']==row['countmin'] else False,axis=1)
    
    TP = comparisonDataFrame[comparisonDataFrame['countminAcc'] == True].count()[0]
    acc = TP / len(comparisonDataFrame)
    return (comparisonDataFrame,acc) 
    # TODO : Play with different width and depth
    #for 

In [32]:
def streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH,counterMatrix,hosts):
    packets =[]
    with open(CTU_NETFLOW_FILEPATH,"r") as netflow:
            #File is an Argus Unidirectional file created with ra -r so not rly csv. 
            #Let's parse by hand.
            for i,packet  in enumerate(netflow):
                #skip header
                if i == 0:
                    continue
                packet = packet.rstrip('\n')
                split_packet = packet.split("\t")
                if "->" in split_packet:
                    split_packet.remove("->")

                if "" in split_packet:
                    split_packet.remove("")

                ## Take all the ports from the ips

                if ":" in split_packet[3]:
                    split_packet[3]= split_packet[3].split(":")[0]
                if ":" in split_packet[4]:
                    split_packet[4]= split_packet[4].split(":")[0]
                
                split_packet = [e for e in split_packet if e]
                if  split_packet[4] not in hosts:
                    print(split_packet[4])
                    update(counterMatrix,str(split_packet[4]))

                packets.append(split_packet)

                if i == amount_of_packets:
                    break
    return packets

In [ ]:
accuracy = 0
amount_of_packets= 10000
hosts=["147.32.84.165","147.32.84.170","147.32.84.134","147.32.84.164","147.32.87.36","147.32.80.9","147.32.87.11"]
for w in range(100,10000,100):
        d = 2
        counterMatrix= matrixCreation(w,d)
        dfStream = streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH,counterMatrix,hosts)
        df = pd.DataFrame(dfStream,columns=COLUMNS)
        _,newAccuracy = determineAcc(df)
        if newAccuracy > accuracy :
            accuracy = newAccuracy
            bestW = w 
print("BEST W")        
print(bestW)

147.32.86.187
2
84
83.208.193.123
27
89
74.125.108.243
35
29
147.32.84.14
19
1
147.32.84.59
14
73
217.168.211.184
75
21
147.32.84.59
14
73
88.103.40.117
53
7
173.9.132.155
54
80
84.242.66.38
73
47
147.32.84.242
54
69
89.233.174.25
5
23
17.174.2.27
79
62
17.174.2.27
79
62
147.32.85.7
7
66
147.32.84.162
18
67
147.32.80.13
45
4
147.32.86.44
27
93
41.137.23.48
53
44
147.32.86.92
29
9
78.80.125.89
27
32
147.32.84.59
14
73
195.23.102.206
24
82
147.32.86.74
31
86
207.200.96.138
52
88
147.32.86.176
66
61
147.32.84.171
69
60
79.126.149.52
80
85
89.185.253.133
46
88
147.32.85.123
36
20
62.44.1.26
32
98
147.32.84.166
69
35
147.32.84.166
69
35
41.153.46.85
59
27
147.32.84.3
36
69
78.80.14.227
72
12
147.32.85.18
29
56
147.32.84.229
58
45
203.82.92.92
32
94
147.32.87.20
26
16
147.32.86.110
55
59
74.125.232.217
60
66
147.32.86.186
18
13
38.107.110.22
48
62
147.32.84.2
58
50
121.54.41.180
94
23
62.245.67.161
34
7
147.32.3.51
29
34
147.32.87.245
94
14
147.32.84.69
35
94
87.230.74.43
44
24
147.32.87.245

61
151.27.148.110
15
7
211.76.78.24
79
78
147.32.84.179
18
3
82.100.63.250
75
43
66.220.151.83
65
27
147.32.87.30
15
25
74.125.232.197
18
45
147.32.84.22
40
57
147.32.85.56
2
48
212.233.178.81
86
78
147.32.86.165
46
49
147.32.84.59
14
73
147.32.84.59
14
73
69.167.127.68
50
80
147.32.85.23
82
97
147.32.86.165
46
49
74.125.79.100
42
53
147.32.86.165
46
49
147.32.86.165
46
49
147.32.86.20
26
21
188.138.84.239
45
25
147.32.84.184
52
1
194.228.44.198
15
42
147.32.84.59
14
73
69.63.180.46
28
93
147.32.84.59
14
73
147.32.84.59
14
73
188.138.84.239
45
25
85.71.203.162
48
54
147.32.84.59
14
73
95.168.199.193
65
84
147.32.84.229
58
45
147.32.86.252
84
36
94.68.71.177
18
71
147.32.84.59
14
73
147.32.87.48
5
1
203.82.92.92
32
94
89.187.142.41
21
78
147.32.84.151
37
43
94.100.187.24
35
43
108.6.32.7
99
8
204.186.11.249
30
19
195.113.232.75
29
82
147.32.84.59
14
73
195.113.232.75
29
82
147.32.84.59
14
73
111.252.244.242
85
75
193.170.140.70
36
3
193.170.140.70
36
3
66.220.153.19
1
40
147.32.84.22
40

147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.85.56
2
48
147.32.84.59
14
73
87.126.189.2
0
47
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.85.56
2
48
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14
73
147.32.84.59
14

52
63
147.32.86.186
18
13
147.32.84.59
14
73
147.32.84.14
19
1
217.168.211.184
75
21
147.32.84.59
14
73
173.9.132.155
54
80
147.32.86.186
18
13
213.232.93.3
60
36
147.32.84.59
14
73
88.103.40.117
53
7
147.32.84.166
69
35
147.32.86.44
27
93
41.137.23.48
53
44
147.32.84.229
58
45
77.76.148.126
71
49
147.32.86.92
29
9
78.80.125.89
27
32
147.32.86.118
65
67
147.32.84.229
58
45
95.87.154.222
99
16
147.32.84.162
18
67
147.32.86.44
27
93
195.68.214.178
5
55
141.34.68.65
41
27
125.232.97.43
24
40
78.106.238.95
98
33
147.32.85.123
36
20
147.32.86.165
46
49
192.35.241.84
41
97
147.32.85.7
7
66
41.153.46.85
59
27
147.32.86.165
46
49
147.32.84.59
14
73
147.32.84.171
69
60
79.126.149.52
80
85
62.44.1.26
32
98
147.32.80.13
45
4
147.32.80.13
45
4
147.32.84.229
58
45
141.21.42.6
28
84
147.32.84.83
54
60
147.32.84.138
77
48
194.108.204.27
24
69
147.32.85.100
90
47
74.125.232.215
28
43
147.32.86.110
55
59
207.200.96.138
52
88
147.32.85.33
57
74
62.24.64.52
42
63
147.32.85.124
14
17
147.32.84.59
14
73
14

36
147.32.86.194
83
92
147.32.86.74
31
86
147.32.85.100
90
47
147.32.84.59
14
73
147.32.84.59
14
73
147.32.3.51
29
34
74.125.232.200
87
14
147.32.86.187
2
84
147.32.84.0
64
53
74.125.232.198
71
69
74.125.232.198
71
69
74.125.232.194
77
89
147.32.86.187
2
84
147.32.86.187
2
84
147.32.86.187
2
84
147.32.84.46
7
3
147.32.85.5
26
94
88.101.110.170
0
93
74.125.232.217
60
66
115.186.170.139
45
57
86.40.101.227
80
31
89.233.161.73
94
7
147.32.86.187
2
84
64.134.178.57
93
18
147.32.87.126
27
58
147.32.87.2
25
20
147.32.84.138
77
48
147.32.84.138
77
48
147.32.84.138
77
48
147.32.85.25
33
94
147.32.85.25
33
94
147.32.84.138
77
48
147.32.84.138
77
48
147.32.85.25
33
94
147.32.85.25
33
94
147.32.85.25
33
94
147.32.85.25
33
94
147.32.85.25
33
94
147.32.84.138
77
48
147.32.86.183
82
27
149.5.45.7
96
98
147.32.85.109
36
4
147.32.87.22
24
60
147.32.3.51
29
34
74.125.232.206
38
34
74.125.232.206
38
34
74.125.232.206
38
34
74.125.232.206
38
34
74.125.232.206
38
34
74.125.232.206
38
34
147.32.84.229
58
4

...

14
273
131.111.208.210
136
63
147.32.84.162
118
67
147.32.84.229
58
45
147.32.84.59
14
273
147.32.84.59
14
273
224.0.0.13
1
190
147.32.86.186
18
213
147.32.86.99
279
26
147.32.84.229
58
45
131.111.208.210
136
63
195.113.232.89
200
297
147.32.84.59
14
273
64.4.34.87
221
218
147.32.80.13
145
204
147.32.87.40
39
51
72.208.146.113
119
107
74.125.39.125
269
7
147.32.86.20
26
121
193.93.174.21
121
48
193.93.174.21
121
48
199.47.217.173
190
233
74.125.232.214
164
95
147.32.85.18
229
156
147.32.86.165
146
249
79.50.118.87
111
9
147.231.6.226
83
36
147.231.6.226
83
36
199.47.216.174
296
77
213.146.189.202
123
74
41.137.23.48
53
244
209.85.148.147
86
69
147.32.85.119
227
98
147.32.86.44
227
293
147.32.84.111
84
200
147.32.85.60
288
101
80.250.0.34
116
73
80.250.0.34
116
73
208.88.186.5
266
259
192.168.253.41
51
2
209.85.148.104
211
282
147.32.84.59
14
273
74.125.39.125
269
7
147.32.85.119
227
98
188.95.61.42
91
285
147.32.85.18
229
156
147.32.84.111
84
200
147.32.86.110
155
259
147.32.84.59
14
2

67
194.79.52.199
54
36
147.32.86.53
250
187
147.32.86.53
250
187
147.32.86.53
250
187
74.125.232.201
238
129
194.79.52.204
90
39
147.32.86.53
250
187
195.168.10.171
3
12
194.79.52.204
90
39
194.79.52.196
43
143
194.79.52.199
54
36
194.79.52.199
54
36
194.79.52.199
54
36
194.79.52.199
54
36
147.32.86.53
250
187
194.79.52.199
54
36
147.32.86.53
250
187
147.32.86.53
250
187
147.32.86.53
250
187
147.32.86.53
250
187
147.32.86.53
250
187
147.32.86.53
250
187
194.79.52.199
54
36
147.32.86.53
250
187
194.79.52.199
54
36
147.32.86.53
250
187
147.32.86.53
250
187
147.32.86.53
250
187
195.168.10.171
3
12
147.32.86.53
250
187
194.79.52.194
11
34
194.79.52.194
11
34
147.32.84.59
14
273
74.125.232.192
167
33
147.32.86.53
250
187
74.125.79.100
242
153
95.168.199.193
165
84
147.32.84.184
52
201
147.32.84.162
118
67
147.32.84.162
118
67
194.79.52.204
90
39
194.79.52.204
90
39
147.32.84.59
14
273
212.161.8.36
264
19
74.125.232.198
271
169
147.32.85.84
210
261
213.239.193.195
297
174
147.32.86.101
34
28

212.87.14.35
215
294
147.32.86.74
231
286
199.47.217.149
143
168
129.174.93.11
113
120
147.32.84.59
14
273
64.215.254.120
126
102
147.32.86.44
227
293
147.32.85.25
133
194
147.32.85.18
229
156
147.32.86.165
146
249
141.30.208.96
58
183
147.32.84.59
14
273
147.32.84.59
14
273
147.32.84.59
14
273
147.32.84.59
14
273
147.32.84.59
14
273
147.32.84.59
14
273
147.32.85.75
9
200
74.125.232.218
229
244
147.32.84.59
14
273
64.215.254.120
126
102
147.32.85.74
209
117
147.32.84.59
14
273
62.245.66.71
106
78
147.32.84.59
14
273
82.208.56.89
195
163
147.32.86.96
227
71
147.32.84.59
14
273
111.192.177.24
26
68
111.248.69.60
274
137
220.239.64.239
156
32
59.175.35.34
160
35
195.113.144.245
254
110
147.32.87.22
24
260
147.32.3.51
29
134
74.125.232.200
87
114
147.32.84.59
14
273
209.85.149.189
131
250
147.32.86.44
227
293
147.32.84.59
14
273
205.188.17.70
158
103
195.113.232.96
199
258
147.32.84.59
14
273
74.125.232.213
112
107
239.255.255.250
123
223
74.125.232.201
238
129
147.32.84.25
242
2
147.32.86

14
273
147.32.86.165
146
249
147.32.86.165
146
249
147.32.85.25
133
194
147.32.85.25
133
194
147.32.84.229
58
45
147.32.85.124
214
17
109.123.209.192
147
289
147.32.85.124
214
17
109.123.209.192
147
289
109.123.209.192
147
289
109.123.209.192
147
289
147.32.85.124
214
17
109.123.209.192
147
289
147.32.85.124
214
17
147.32.85.124
214
17
109.123.209.192
147
289
147.32.85.124
214
17
147.32.85.124
214
17
147.32.85.124
214
17
147.32.86.165
146
249
147.32.84.13
127
141
109.123.209.241
177
254
147.32.85.124
214
17
147.32.86.239
285
216
147.32.84.138
177
148
147.32.84.138
177
148
147.32.85.124
214
17
188.120.32.60
290
238
147.32.84.138
177
148
147.32.85.124
214
17
147.32.84.138
177
148
195.216.243.17
261
16
147.32.84.138
177
148
147.32.84.138
177
148
147.32.84.138
177
148
147.32.84.59
14
273
147.32.85.124
214
17
147.32.85.124
214
17
147.32.84.59
14
273
147.32.192.34
174
182
147.32.85.95
281
140
147.32.84.229
58
45
85.65.127.211
154
258
89.233.174.37
268
145
147.32.84.59
14
273
80.83.61.107
2
1

17
147.32.85.56
102
148
212.233.178.81
86
78
147.32.84.110
48
129
Broadcast
70
105
89.233.174.37
268
145
147.32.86.186
18
213
147.32.84.59
14
273
147.32.84.59
14
273
209.85.148.105
78
200
147.32.84.59
14
273
78.141.181.222
205
187
131.111.23.68
34
144
147.32.86.92
229
209
147.32.84.59
14
273
147.32.84.59
14
273
88.86.102.50
216
4
147.32.84.59
14
273
74.125.232.216
4
248
147.32.84.59
14
273
62.44.12.24
148
185
147.32.84.59
14
273
147.32.86.92
229
209
213.29.198.144
6
290
147.32.86.20
26
121
80.239.207.156
129
251
147.32.85.15
267
14
147.32.84.138
177
148
145.116.233.184
71
239
147.32.84.59
14
273
194.108.204.27
224
169
147.32.84.171
269
60
80.239.207.156
129
251
147.32.84.59
14
273
94.100.187.24
135
243
147.32.84.59
14
273
147.32.84.59
14
273
147.32.84.59
14
273
94.14.70.74
154
259
147.32.84.59
14
273
147.32.84.59
14
273
147.32.84.59
14
273
151.75.197.68
166
54
147.32.84.59
14
273
147.32.84.59
14
273
151.75.197.68
166
54
147.251.48.140
101
54
147.32.84.5
268
287
147.32.84.59
14
273
147.

In [ ]:
newAccuracy

In [ ]:
bestW

In [ ]:
# w is the best value obtained in the previous calculations
w = bestW
counterMatrix= matrixCreation(w,d)
dfStream = streamFunction(amount_of_packets,CTU_NETFLOW_FILEPATH,counterMatrix,hosts)
df = pd.DataFrame(dfStream,columns=COLUMNS)
comparisonDF, finalAccuracy = determineAcc(df)

In [ ]:
#comparisonDataFrame.sort_values('True Values',ascending=False)

In [ ]:
print(finalAccuracy)

In [ ]:
infectedNode= "147.32.84.165"

In [ ]:
comparisonDF.sort_values('True Values',ascending=False)

# Comparison between Sampling task and Sketching task
    - Compare space
    - Compare time
    - Compare accuracy

## Sampling task
    - Each entry generate one time random value rn € U [ 0 ,1 ]